# Capitol Hill Baby Sitting Co Op

Ok, now consider this situation.
We are setting up a baby sitting co op. We are a bunch of families that plan to exchange babysitting services (I baby sit your kids tonight, you babysit mine tomorrow).
We want things to be as fair as possible.
We come up with this idea: we can use some some _vouchers_! Every family is initially given a bunch of vouchers. Whenever family A babysits B's kids, B gives A one voucher. This ensures that nobody exploits or freerides the system too much.
Ok, but now we want to implement these vouchers _digitally_! A doesn't have to pass over to B any germ ridden piece of paper or things, but they want to be able to settle things with the PC.
How can we do that?

## Solution 1: freely accessible table

The first idea is: we set up a shared document, or datastore, in which we represent a table. Or, even better, a key -> value structure.

In [2]:
balances = {"A":10, "B":10, "C":10}

The table is freely accessible, and anyone can modify it. We can think something like a shared google spreadsheet, such as https://docs.google.com/spreadsheets/d/1YhbE_U33mx3gOjPFL9Jd7NhrT-5o3EAvqEsxkRUiHzI/edit?usp=sharing.
Or, we could set up an http REST system, that accepts reading and modification of any key. Or we could use an already existing key value store (such as redis). I A wants to give B one her vouchers, he does something like this:


In [10]:
# A wants to give B one voucher
from redis import Redis
client = Redis()

In [11]:
client.get("A")

b'10'

In [12]:
client.get("B")

b'10'

In [13]:
client.set("A",9)
client.set("B",11)

True

Ok, does this system work? Is it reliable? What are our _trust assumptions_?

1. we have to trust that all the participants behave correctly
2. we have to trust that the redis server is always on

Are these conditions alright? It depends, of course.

What if the first condition is unrealistic? Can we modify/refine the system to weaken condition 1?

## Solution 2: administred data storage

We could elect/select an administrator. The administrator is the only one who can write on the table/database/keyvalue store.
The administrator receives mandate from the user A to transfer some of his vouchers to  user B, and performs the transfer (if A has enough vouchers).
Of course, the administrator has to be able to identify users, which he can do in many, variously effective, ways.
What is our trust assumptions, now? Or, put it in another way, what can users/actors do to compromise the system?
1. the administrator can:
    a. arbitrarily change the balance of any user
    b. keep users from transfering their vouchers (censoring them)
2. the operator of the database can shut down the database, arbitrarily change balances, and censor the admin from operating on data

We can of course automatize this solution, and we obtain an accounting application, like the one of our bank!

Other problems:
* can the administrator prove that A wanted to transfer vouchers to B? (non repudiation)

## Solution 3: signed transactions

We can limit the power of the administrator using _digital signatures_. Now the administrator, instead of identifying users, accepting their transfer mandates, and then execute them, receives _signed mandates_, like bank checks.
A mandate is a signed piece of data, like:
`{from: "A", to: "B", amount: "2"}, signature: "..."`

The administrator then receives the mandate, checks the signature, check the sender balance, and if it's alright, he changes the balance table.

But now we also need to know the public key of each user, so the balance table will look like this:

In [18]:
balances = [["name","pubkey","amount"],
            ["A","0x71f589260c216054",2],
            ["B","0x9b2416192001cb42",3]]

Now, the administrator cannot arbitrarily take 1 voucher away from B and give it to A, because B would notice, and could ask him to prove that he mandated that transfer.

But the administrator can always create voucher out of nowhere! He can raise A's balance by 10, and no one could notice. (no he couldn't, because the balance table is public, and anybody would notice am increase in the total number of vouchers)

What can the administrator still do? Well, he could "undo" a transfer. For example, if A mandates a transfer to B, the administrator could execute it, and then, after a couple of days, in agreement with A, could revert it. B won't be able to prove that he is entitled to the vouchers that he received. How can we solve this?

Well, we could render the mandates list public! When a user wants to perform a transfer, he creates and signs a transaction, and sends it to the administrator. The administrator then checks if the transaction is valid (i.e., if the signature is correct, and if the balance is enough), and updates the balance table.

What are the trust assumptions, here?

1. That the registry of all the transactions is always available, and that cannot be modified

Infact, the controller of registry server could always take away some transaction. If I am looking at the registry, how do I know whether it contains all the transactions up to now?

[comment]: # (insert example)

We need a way to make the transactions registry not tamperable, immutable.

Ah, one more thing: since every user name here is associated with a public key, why not using public keys to identify the owner of a balance? This way, we would have a 

## Solution 4: linked chain

We could do something like this: when a user creates a transaction, it also includes a reference to the previous transaction, in the form of a hash:

```
[
    ["from":"0x71f589260c216054", "to":"0x9b2416192001cb42", "signature":"0x643ac2fdaf7f10399ca32fc3ccb334d8", "previous":"0xe0c90d2de2d35b0a513b17a82e5b55d2"],
    ["from":"0xe27f81ffb2661418", "to":"0x9be156e2d3c5011", "signature":"0x2f5f83dd30f5e3beee5fa45a9c561729", "previous":"0x1322d57171d553008fbe630b2e8518e3"]
    ...
```

In this way, all the transactions are chained. If the provider of the registry takes away one transaction, that can be detected by anyone.

But what happens if a user sends two different transactions, that "point" to the same previous? What shall the administrator do? Keep one and discard the other? What if a user transfers one voucher to a user X, obtain a babysitting service, and then, colluding with the administrator, creates another conflicting transaction in which he transfers the voucher to someone else (maybe to himself in disguise with another newly created public key?). This would be called "double spending"

So our trust assumption now is:

1. the administrator doesn't collude with a user to let him double spend, substituting a transaction

# Solution 5: distributed registry

To remove the dipendency from the administrator, we could _distribute_ the transactions registry. It could work like this:
1. whoever wants, joins a network in which users broadcast transactions. We will call these actors _nodes_
2. whenever a node receives a new transaction, he appends it to the registry

In this way, we would have many copies of the registry, distributed among different nodes.
But we still have some problems:

1. if a user wants to produce a transaction, he has to know which is the latest transaction in the registry, to include its hash.

What happens if two users produce a transaction that has the same previous? One idea would be to chose one of the two, and then the user that produced the discarded one regenerates his. But how to choose?

# Solution 6: validated registry

A solution could be: users don't include, in the transaction, the hash of the previous one. They just include data and signature, and send them to the network, where they are collected into a pool. Then, an actor, that we will call _validator_, at regulary intervals picks a certain number of valid transactions, embeds them in a _block_, links it to the previous one, and send it to the nodes, that append it to their registry. Note that now, if the administrator emits two conflicting blocks, it is possible for anyone to prove that, and to nail him to his responsabilities.

Question: how often should the validator produce blocks? Is there a lower limit? How big should blocks be? Should there be a upper limit? 

But we just reintroduced a single player! What happens if he crashes? Or if he stops accepting transactions? Or if he decides to systematically not include transactions coming from a certain user/publickey? 

# Solution 7: permissioned blockchain

The solution here can be quite simple: let's distribute honor and burden of validation across a set of players. Each one of these takes turn in picking the transactions to insert in the next block. The block then has to be validated by some majority of the validators, and signed by them. Once the new block has been validated, it is broadcast to all nodes. In this way, the system becomes fault tolerant, also in a Byzantine way.

[comment]: # (Insert analysis in terms of CAP properties. See e.g. https://eprints.soton.ac.uk/415083/2/itasec18_main.pdf)

This is called a _permissioned blockchain_. It is the consensus mechanism proposed and used in many applications, such as Libra (now Diem) and institutional blockchains such as EBSI.

They are called _permissioned_ because if you want to play as a validator, you must have permission (the network is defined by the identity of the validators).

This can be perceived as a problem: what happens if the validators collude? or if they are forced to behave some way?

Can we envision a system in which we don't have to rely on the honesty of a defined set of players?

Satoshi Nakamoto, a mysterious entity (a single person? a collective? to date, no one knows), had an idea on how to do that (https://bitcoin.org/bitcoin.pdf)

# Solution 8: permissionless proof of work blockchain

Here the idea is: anybody could want to be the next validator, and chose the transactions to be appended. Me, you, everybody. So, how do we choose? We have to find a way to agree on who's the next validator. Note that here we aim to _eventual consistency_.

Satoshi's idea is this one: do you want to create the next block in the blockchain? Ok. Pick the pending transaction you want, and put them into a block. Don't forget to add the hash of the previous block. Now, the system asks you to do one thing, to validate the block: you have to do some work, spend some energy _on the block that you are proposing_. The fact that you spent energy must be evident to anyone, looking at the block. How can we achieve that? But with proof of work, of course! The rule is that you have to find a nonce, that, appended to the block data, renders some property of the hash true (like for instance, having 10 leading '0's). When you find it, you broadcast the validate block to the network, and the nodes append it to 
This means that being a validator will cost you (money, thus energy). Once you found 
And why would you want to do that? Because the system will reward you: in your block you have to insert a special transaction, in which you transfer, to an address of your choice, a number of newly created vouchers. If vouchers are worth anything, this can be a in incentive for you. Please note that the newly minted voucher is yours _only_ if your block will become part of the chain.

What happens if two candidate validators both validate their different blocks?
Say both A and B created the 100th block (we'll call it block of _height_ 100). They both validated their blocks at roughly the same time. They broadcast the block to the nodes. Which block should nodes accept?
What happens is that the system is in a state of inconcistency. This state is called a _fork_. We have to possible blocks of height 100. How does the system resolve this state of things?

First of all, with a consensus rule, that says: if there are two forks, the longest is the "right" one. This is a simple relationship among different possible versions of the chain of blocks, that can be tested by anyone.

Candidate validators (from now on _miners_) of block 101 will have to choose which block to consider the right one, because they'll have to include its hash in their block, and then spend work on it. Now: a miner has all the interest to choose the block that will be considered, in the future, the right one, that is the same that will chosen by all the others. Otherwise, his block will become part of a dead branch of history, and his reward will remain dead letter. 

This has an effect: it renders forks intrinsically _unstable_. As soon as one of the two branches exceeds the other in length, that one will be considered the right one, and all the miners will concentrate their work on it, accelerating it even more, and leaving the others behind.

This is the protocol of Bitcoin, where instead of babysitting vouchers we have "tokens" that are called bitcoin.

So, what kind of Byzantine attacks can the network withstand?
* if a user wants to double spend his vouchers, he has to create a fork. But forks in the past are exceedingly difficult to sustain, because the attacker would have to catch up with the current size of the chain.

This renders the chain very difficult to change. And the more the blocks are old (or _finalized_), the more they are hard to change. Changing block 100, when the current valid chain has height 200, would mean validating 100 blocks+the ones that are validated on the main chain in the meanwhile.

Questions:
* how does the number/computational power of miners affect the blocktime?
* what can we say about the number of existing tokens?
